In [1]:
import networkx as nx
import numpy as np

import input_functions as inp
import tsp_routines
from clustering.funcs import k_cluster
from clustering.funcs import best_dropoff
from input_functions.funcs import create_new_graph

In [2]:
def graph_from_input(filename):
    '''graph_from_input(str) --> nx.graph G, np.ndarray[int] homes, dict locToIndex 
    Returns a graph created by reading the input file, with integer vertex labels
    Returns list of the home indices
    Returns a map from integer to the name associated with that node'''
    with open(filename, 'r') as f:
        G = nx.Graph()
        
        locToIndex = {} # maps location name to its index number
        homes = []
        lines = f.readlines()
        
        numLocations = int(lines[0])
        numTAs = int(lines[1])
        locations = lines[2].split()
    
        i = 0
        assert len(locations) == numLocations, "Number of locations must match specified value"
        for loc in locations:
            G.add_node(i)
            locToIndex[loc] = i
            i += 1
            
        TAhomes = lines[3].split()
        assert len(TAhomes) == numTAs, "Number of TA homes must match specified value"
        for home in TAhomes:
            homes.append(locToIndex[home])
        
        homes.insert(0, locToIndex[lines[4].strip()])
        
        row = 0
        for line in lines[5:]:
            line = line.split()
            for col in range(len(line)):
            
                if line[col] != 'x':  
                    G.add_edge(row, col)
                    weight = float(line[col])
                    G[row][col]['weight'] = weight
            row += 1
            
        indexToLoc = {v: k for k, v in locToIndex.items()}
        return G, homes, indexToLoc

In [3]:
def eval_cost(G, path, dropoffs):
    '''
    eval_cost(nx.graph, np.ndarray, dict) -> int
    path is a list of integers that we follow in the car 
    dropOffs is a dictionary (int -> [home, home, ...] )
    '''
    assert isinstance(G, nx.Graph) , "G must be a graph"
    assert isinstance(path, np.ndarray) , "path must be an array of integers"
    assert isinstance(dropoffs, dict) , "dropoffs must be a dictionary mapping node to homes"
    
    cost = 0
    prevNode = path[0]
    for node in path[1:]:
        cost += 2/3 * G[prevNode][node]['weight'] # weight of the edge from previous to next node
        prevNode = node
        
        for home in dropoffs[node]:
            cost += nx.astar_path_length(G, node, home)
                
    return cost

In [4]:
def map_dict_entries(input_dict,nodemapper):
    """
    maps the keys and values of the input dict using the nodemapper dict.
    """
    output_dict = dict()
    for key in input_dict.keys():
        mapped_key = nodemapper[key]
        output_dict.update({mapped_key:[]})
        for vertex in input_dict[key]:
            output_dict[mapped_key].append(nodemapper[vertex])
    return output_dict

def add_vertex_to_clusters(clusters,vertex):
    """
    add the given vertex to each cluster.
    Input:
    clusters - dict where the keys are vertices which are cluster centers and the values are a list of 
                vertices belonging to this cluster
    vertex - the vertex to be added to each list in `clusters`
    """
    for key in clusters:
        clusters[key].append(vertex)
        
def get_dropoff_vertices(clusters):
    best_dropoffs = []
    for key in clusters:
        dropoff = best_dropoff(G,clusters[key])
        best_dropoffs.append(dropoff)
    return best_dropoffs
        
def solve_by_clustering(graph,homes,source,num_clusters):
    """
    return the route to be followed by the car as it drops off TAs.
    Inputs:
    graph - input graph
    homes - list of vertices in `graph` that are marked as homes
    source - vertex in `graph` that is the start and end of the path followed by the car
    num_clusters - the number of clusters to be used to group the homes together
    """
    homes_subgraph = tsp_routines.complete_shortest_path_subgraph(graph,homes)
    home_clusters = k_cluster(homes_subgraph,num_clusters)
    # The source vertex is added to each of the clusters before determining the best dropoff location.
    # This is done so that vertices that are closer to the source are given higher preference as dropoff points.
    add_vertex_to_clusters(home_clusters,source)
    dropoff_vertices = get_dropoff_vertices(home_clusters)
    # Add the source to the dropoff vertices
    dropoff_vertices.append(source)
    # Get rid of any repeating entries in the dropoff vertices
    dropoff_vertices = list(set(dropoff_vertices))
    # Construct the fully connected sub-graph with the dropoff vertices 
    # on which TSP is computed
    dropoff_subgraph = tsp_routines.complete_shortest_path_subgraph(graph,dropoff_vertices)
    tsp_route = tsp_routines.metric_mst_tsp(dropoff_subgraph,source)
    return tsp_route
    

In [21]:
def check_cost(G, route, homes):
    carcost = 0
    workingnode = route[0]
    for node in route[1:]:
        carcost+=(2/3)*nx.dijkstra_path_length(G, workingnode, node)
        workingnode = node
        
    ta_cost = 0
    where_get_off = dict()
    for home in homes:
        local_cost = np.inf
        for node in route:
            l = nx.dijkstra_path_length(G, home, node)
            if l < local_cost:
                local_cost = l
                bestnode = node
            else:
                continue
        ta_cost += local_cost
        try:
            where_get_off[bestnode].append(home)
        except KeyError:
            where_get_off.update({bestnode:[home]})
    return carcost, where_get_off

In [22]:
def cluster_solver(filename,kiterations = 'all'):
    """returns the best route and its cost and where each ta gets off, found by decreasing nclusters from nhomes to 2"""
    G, homes, node_to_name_map = graph_from_input(filename)
    k = len(homes) #how many clusters initially
    cost = np.inf
    out_route = []
    if kiterations == 'all':
        krange = range(k-2)
    else:
        krange = range(kiterations)
    for i in krange:
        #assumed that homes[0] is the source
        tsp_route = solve_by_clustering(G,homes,homes[0],k)
        tempcost, where_get_off = eval(G, tsp_route, homes)
        if tempcost<cost:
            cost = tempcost
            out_route = tsp_route
            dropoffloc_dict = where_get_off
        k-=1
    return out_route, cost, dropoffloc_dict

In [23]:
filename = './inputs/20_50.in'
G, homes, node_to_name_map = graph_from_input(filename)

In [34]:
route, cost, dropoffs = cluster_solver(filename, kiterations=1)

In [38]:
tsp_route = solve_by_clustering(G,homes[1:],homes[0],26)

In [40]:
G.has_edge(37,13)

False

In [35]:
print(route, cost)

[21, 10, 7, 19, 29, 37, 13, 8, 11, 23, 36, 9, 46, 41, 32, 31, 24, 6, 47, 42, 2, 34, 17, 25, 1, 27, 21] 5163591867.49498


In [37]:
nx.dijkstra_path(G,37, 13)

[37, 35, 13]

In [26]:
eval_cost(G, np.array(route), dropoffs)

KeyError: 13